### TODOs
- Adjust the object scoring function to:
    - score directly on the 3DObject class once it is created from Matcher
- Write a plotting function for ObjectInstance debugging

In [102]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.io as pio
import cv2

# Ensure Plotly is set up to work with Jupyter notebooks
pio.renderers.default = 'notebook'

# Hack the path for now, deal with this later
cwd = '/teamspace/studios/this_studio/letsdoit'
if (cwd not in sys.path):
    sys.path.append(cwd)

from masks_finder import MasksFinder, unrotate_masks, unrotate_bboxes
from masks_matcher import MasksMatcher
from clip_retriever import ClipRetriever
from dataloader.dataloader import DataLoader
from object_scorer import ObjectScorer
from letsdoit.utils.object_instance import ObjectInstance, initialize_object_instances, plot_instances_3d
from letsdoit.utils.misc import select_ids

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
path_dataset = '/teamspace/studios/this_studio/datasets'
ASSET_TYPE = 'wide'
loader = DataLoader(path_dataset, split='dev')
retriever = ClipRetriever()
masks_finder = MasksFinder()
visit_ids = loader.visit_ids
video_ids = loader.get_video_ids(visit_ids[0])

visit_id = visit_ids[0]
video_id = video_ids[0]
# Get both original and upright-rotated images and depths as outputs
images, images_rotated, image_paths, intrinsics, poses, orientations = loader.get_images(visit_id, video_id, asset_type=ASSET_TYPE, sample_freq=1)
depths, depths_rotated, depth_paths, _, _, _ = loader.get_depths(visit_id, video_id, asset_type=ASSET_TYPE, sample_freq=1)
# Retriever receives upright-rotatead images as inputs
retriever.generate_image_features(images_rotated)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
<All keys matched successfully>


Loading rgb frames from visit 420683 and video 42445132: 100%|██████████| 159/159 [00:07<00:00, 20.76it/s]
Loading depth frames from visit 420683 and video 42445132: 100%|██████████| 159/159 [00:05<00:00, 28.44it/s]


In [89]:
# For an object, get a list of corresponding ObjectInstances
object = 'drawer'
best_indices = retriever.retrieve_best_images_for_object(object)
best_images = select_ids(images, best_indices)
best_images_rotated = select_ids(images_rotated, best_indices)
best_image_paths = select_ids(image_paths, best_indices)
best_intrinsics = select_ids(intrinsics, best_indices)
best_poses = select_ids(poses, best_indices)
best_orientations = select_ids(orientations, best_indices)
best_depths = select_ids(depths, best_indices)
best_depth_paths = select_ids(depth_paths, best_indices)
# Masks we get here as outputs are for the upright-rotated images
image_ids, masks, bboxes, confidences, labels = masks_finder.get_masks_from_imgs(best_images_rotated, object)
# Rotate masks and bboxes back to the rotation of the original image
mask_image_sizes = [best_images_rotated[idx].shape[:-1] for idx in image_ids]
mask_image_orientations = [best_orientations[idx] for idx in image_ids]
masks_unrotated = unrotate_masks(masks=masks, orientations=orientations)
bboxes_unrotated = unrotate_bboxes(bboxes=bboxes, img_dims=mask_image_sizes, orientations=mask_image_orientations)

tensor(0.9478, grad_fn=<MaxBackward1>)
tensor(0.9813, grad_fn=<MaxBackward1>)
tensor(0.9740, grad_fn=<MaxBackward1>)
tensor(0.9459, grad_fn=<MaxBackward1>)
tensor(0.9448, grad_fn=<MaxBackward1>)
tensor(0.9718, grad_fn=<MaxBackward1>)
tensor(0.9824, grad_fn=<MaxBackward1>)
tensor(0.9769, grad_fn=<MaxBackward1>)
tensor(0.9677, grad_fn=<MaxBackward1>)
tensor(0.8786, grad_fn=<MaxBackward1>)
tensor(0.9718, grad_fn=<MaxBackward1>)
tensor(0.9786, grad_fn=<MaxBackward1>)
tensor(0.9649, grad_fn=<MaxBackward1>)
tensor(0.9656, grad_fn=<MaxBackward1>)
tensor(0.9495, grad_fn=<MaxBackward1>)
tensor(0.9896, grad_fn=<MaxBackward1>)
tensor(0.9648, grad_fn=<MaxBackward1>)
tensor(0.9788, grad_fn=<MaxBackward1>)
tensor(0.9703, grad_fn=<MaxBackward1>)
tensor(0.9645, grad_fn=<MaxBackward1>)
tensor(0.9518, grad_fn=<MaxBackward1>)
tensor(0.9730, grad_fn=<MaxBackward1>)
tensor(0.9319, grad_fn=<MaxBackward1>)
tensor(0.9584, grad_fn=<MaxBackward1>)
tensor(0.9786, grad_fn=<MaxBackward1>)
tensor(0.9494, grad_fn=<M

  0%|          | 0/35 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/modeling_utils.py:768: FutureWarning:

The `device` argument is deprecated and will be removed in v5 of Transformers.

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning:

torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None

100%|██████████| 35/35 [00:41<00:00,  1.18s/it]


In [144]:
dict_object_instances = {'images': select_ids(best_images, image_ids),
                         'image_names': [Path(img_path).name.replace('.png', '') for img_path in best_image_paths],
                         'depths': select_ids(best_depths, image_ids),
                         'bboxes': bboxes_unrotated,
                         'masks': masks_unrotated,
                         'labels': labels,
                         'confidences': confidences,
                         'intrinsics': select_ids(best_intrinsics, image_ids),
                         'extrinsics': select_ids(best_poses, image_ids),
                         'orientations': select_ids(best_orientations, image_ids)}

object_instances = initialize_object_instances(**dict_object_instances)

In [147]:
plot_instances_3d(object_instances[0:9], subsample=True)